In [1]:
import pandas as pd
import sqlite3 as db
import numpy as np
'''Import .py modules with processed data: readcsv contains Kaggle data, WebScrapingAtlantaEvents contains Discover Atlanta event data,
and APICALL contains Budget Car Rental data'''
import readcsv
import WebScrapingAtlantaEvents
import APICALL

In [2]:
# Creates a blank db if one does not exist, otherwise opens connection to rentaldb, and drops the tables Kaggle, API, and Webscrape
conn = db.connect('rentaldb')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS Kaggle")
c.execute("DROP TABLE IF EXISTS API")
c.execute("DROP TABLE IF EXISTS Webscrape")

In [5]:
#Runs the sort_lux_df function from the readcsv import. This function reads the CarRentalDataV1 csv data into a dataframe
#sorts it based on location (Atlanta), then filters the vehciles to only show luxury brands/models.
csvdf = readcsv.sort_lux_df()
#This injects the sort_lux_df into the Kaggle table of the rentaldb 
csvdf.to_sql('Kaggle', conn, if_exists='replace')

#Runs the get_events function from the WebScrapingAtlantaEvents import. This function scrapes Discover Atlanta, 
#and formats the events
webdf = WebScrapingAtlantaEvents.get_events('2022-11-20', '2023-02-03')
#This injects the get_events data into the Webscrape table of the rentaldb 
webdf.to_sql('Webscrape', conn, if_exists='replace')

#Runs the api_request function from APICALL. This function calls the API, and reads the data as JSON, then turns it into a df
df2 = APICALL.api_request()
#Runs the clean_api_df function from APICALL. This function cleans the df2 dataframe that the api_request function created
apidf = APICALL.clean_api_df(df2)
#This injects the cleaned API data into API table of the rentaldb
apidf.to_sql('API', conn, if_exists='replace')


27

In [6]:
#Verifying the tables are in the rentaldb
csvdf = pd.read_sql_query("SELECT * FROM Kaggle", conn)
apidf = pd.read_sql_query("SELECT * FROM API", conn)
scrapedf = pd.read_sql_query("SELECT * FROM Webscrape", conn)
print('Kaggle Data')
display(csvdf.head())
print('API Data')
display(apidf.head())
print('Webscrape Data')
display(scrapedf.head())

Kaggle Data


,index,fuelType,rating,renterTripsTaken,reviewCount,city,country,latitude,location.longitude,location.state,owner.id,daily_rate,make,model,type,year,airportcity
0,41,GASOLINE,5.00,12,10,Grayson,US,33.906905,-83.960234,GA,1295660,175,Mercedes-Benz,S-Class,car,2015,Atlanta
1,42,GASOLINE,5.00,2,2,Atlanta,US,33.891295,-84.453929,GA,12796797,200,BMW,7 Series,car,2018,Atlanta
2,50,GASOLINE,4.96,152,140,Atlanta,US,33.927684,-84.343055,GA,511103,155,Chevrolet,Corvette Stingray,car,2014,Atlanta
3,54,GASOLINE,5.00,23,21,Atlanta,US,33.740676,-84.358236,GA,8752492,209,Audi,RS 7,car,2014,Atlanta
4,57,ELECTRIC,4.88,18,16,Duluth,US,33.970512,-84.161462,GA,8609349,159,Tesla,Model S,car,2018,Atlanta


API Data


,index,Vehicle_id,Vehicle Name,Category,Transmission,Dropoff City,Base Price,Pickup City
0,212,644881573,Cadillac XTS,Luxury,Automatic,,368.30,
1,140,673928006,Mercedes-Benz GLC,Luxury,Automatic,,367.62,
2,202,636479888,Cadillac XTS,Luxury,Automatic,,354.48,
3,159,674291426,Lincoln Continental,Luxury,Automatic,,352.51,
4,193,646119413,Cadillac XTS,Luxury,Automatic,,342.96,


Webscrape Data


,index,Event,Dates,Location,Description
0,1807,"Garden Lights, Holiday Nights",2022-11-21 00:00:00,"Atlanta Botanical Garden, Inc.","The 12th annual Garden Lights, Holiday Nights ..."
1,1808,Spruill Gallery Holiday Artists Market,2022-11-21 00:00:00,Spruill Gallery,This well-loved Holiday Artist’s Market kicks ...
2,1809,Stem Wine Bar’s November Tastings: Amazing Ame...,2022-11-21 00:00:00,Stem Wine Bar,Join Stem Wine Bar for their November wine tas...
3,1810,Reindeer Road by World of Illumination,2022-11-21 00:00:00,Six Flags WHITE WATER,A merrily magical journey Grab your sled and j...
4,1811,Rudolph the Red-Nosed Reindeer,2022-11-21 00:00:00,Center For Puppetry Arts,A Holiday Classic Returns to the Stage Based o...
